## AndLink and EvaluationLink examples

In [1]:
from opencog.utilities import initialize_opencog
from opencog.atomspace import AtomSpace, types
from opencog.type_constructors import *
from opencog.bindlink import bindlink
from opencog.bindlink import execute_atom

/home/noskill/miniconda3/envs/pmvqa3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'opencog.utilities' does not match runtime version 3.5
  return f(*args, **kwds)
/home/noskill/miniconda3/envs/pmvqa3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'opencog.atomspace' does not match runtime version 3.5
  return f(*args, **kwds)
/home/noskill/miniconda3/envs/pmvqa3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'opencog.type_constructors' does not match runtime version 3.5
  return f(*args, **kwds)
/home/noskill/miniconda3/envs/pmvqa3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'opencog.bindlink' does not match runtime version 3.5
  return f(*args, **kwds)


create atomspace and set it as default

In [2]:
atomspace = AtomSpace()
initialize_opencog(atomspace)

### knowledge base

Color and size are attributes

In [3]:
InheritanceLink(ConceptNode("size"), ConceptNode("attribute"))
InheritanceLink(ConceptNode("color"), ConceptNode("attribute"))

(InheritanceLink
  (ConceptNode "color") ; [4437725137865661335][1]
  (ConceptNode "attribute") ; [8286119068776667766][1]
) ; [15737919214045963885][1]

different types of color and size

In [4]:
InheritanceLink(ConceptNode("small"), ConceptNode("size"))
InheritanceLink(ConceptNode("big"), ConceptNode("size"))
InheritanceLink(ConceptNode("green"), ConceptNode("color"))
InheritanceLink(ConceptNode("red"), ConceptNode("color"))
InheritanceLink(ConceptNode("black"), ConceptNode("color"))

(InheritanceLink
  (ConceptNode "black") ; [3641150941895841241][1]
  (ConceptNode "color") ; [4437725137865661335][1]
) ; [17348854327767872816][1]

animals which have some attributes defined

In [5]:
InheritanceLink(ConceptNode("frog"), ConceptNode("small")).tv = TruthValue(0.8, 0.9)
InheritanceLink(ConceptNode("frog"), ConceptNode("green")).tv = TruthValue(0.7, 0.9)
InheritanceLink(ConceptNode("tiger"), ConceptNode("big")).tv = TruthValue(0.9, 0.9)
InheritanceLink(ConceptNode("tiger"), ConceptNode("red")).tv = TruthValue(0.6, 0.9)
InheritanceLink(ConceptNode("sparrow"), ConceptNode("red")).tv = TruthValue(0.1, 0.9)
InheritanceLink(ConceptNode("sparrow"), ConceptNode("small")).tv = TruthValue(0.8, 0.9)
InheritanceLink(ConceptNode("zebra"), ConceptNode("black")).tv = TruthValue(0.5, 0.9)
InheritanceLink(ConceptNode("zebra"), ConceptNode("small")).tv = TruthValue(0.38, 0.9)

all items are items

In [6]:
InheritanceLink(ConceptNode("frog"), ConceptNode("animal"))
InheritanceLink(ConceptNode("frog"), ConceptNode("animal"))
InheritanceLink(ConceptNode("tiger"), ConceptNode("animal"))
InheritanceLink(ConceptNode("tiger"), ConceptNode("animal"))
InheritanceLink(ConceptNode("sparrow"), ConceptNode("animal"))
InheritanceLink(ConceptNode("sparrow"), ConceptNode("animal"))
InheritanceLink(ConceptNode("zebra"), ConceptNode("animal"))
InheritanceLink(ConceptNode("zebra"), ConceptNode("animal"))

(InheritanceLink
  (ConceptNode "zebra") ; [1533348427618083972][1]
  (ConceptNode "animal") ; [4997541822935110438][1]
) ; [12181097095668208330][1]

let's select all animals and attributes, filtering by some threshold


first define function and EvaluationLink

In [21]:
threshold = 0.6

def apply_threshold(atom):
    if atom.tv.mean < threshold:
        return TruthValue(0, 1)
    return TruthValue(1, 1)

EvaluationLink will accept link    X <- Z  
and return flag if it "matches"

In [22]:
inherits_property = InheritanceLink(VariableNode("X"), VariableNode("Z"))

# all Variables(X and Z) used in EvaluationLink should be grounded somewhere else first!
# eval link which provides filtering
eval_threshold = EvaluationLink(
                    GroundedPredicateNode("py: apply_threshold"),
                    ListLink(inherits_property))

### now build the BindLink

Constrain that Variable "X" inherits from "item"

In [23]:
select_items = InheritanceLink(VariableNode("X"), ConceptNode("animal"))

In [24]:
# using NotLink
Z_is_not_item = NotLink(IdenticalLink(VariableNode("Z"), ConceptNode("animal")))

In [25]:
and_link = AndLink(eval_threshold, inherits_property, Z_is_not_item, select_items)
bindlink_filter = BindLink(and_link,
                           inherits_property)


print(bindlink(atomspace, bindlink_filter))

(SetLink
  (InheritanceLink (stv 0.800000 0.900000)
    (ConceptNode "frog")
    (ConceptNode "small")
  )
  (InheritanceLink (stv 0.600000 0.900000)
    (ConceptNode "tiger")
    (ConceptNode "red")
  )
  (InheritanceLink (stv 0.700000 0.900000)
    (ConceptNode "frog")
    (ConceptNode "green")
  )
  (InheritanceLink (stv 0.900000 0.900000)
    (ConceptNode "tiger")
    (ConceptNode "big")
  )
  (InheritanceLink (stv 0.800000 0.900000)
    (ConceptNode "sparrow")
    (ConceptNode "small")
  )
)



## Exercise: remove Z_is_not_item from condition